### Reproduce of calibration evaluation of Epstemic and Aleatoric Uncertainty

In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob

plt.rcParams['lines.markersize'] = 3
fig, axes = plt.subplots(4, 3, figsize=(12, 9))

result_dir = '../result/calibration/' 
ea_ous = 'epis_alea_ours'
ea = 'epis_alea'
e = 'epis'

'''
    Make sure these dataset has been outputted by `run.sh`
'''
dataset_list =  [ "SMAP_P1",  "NAB_Ambient", "NAB_Texi", "NAB_Machine",  "SMD_machine3-4", "MSL_F7", "MSL_P11", "SMAP_E13", "SMAP_E1", "UCR_InternalBleeding16", "UCR_InternalBleeding17", "SMD_machine2-1"] 

for index, dataset in enumerate(dataset_list):
    row = index // 3
    col = index % 3
    ea_ous_obsers = [[] for i in range(5)]
    ea_obsers = [[] for i in range(5)]
    e_obsers = [[] for i in range(5)]

    exists = any(glob.glob(f'{result_dir}/{dataset}*'))
    if exists == False:
        print(f'{dataset} does not exist, skipping...')
        continue
    for seed in [1,2,3,4,5]:
        df_epis_alea_ours = pd.read_csv(f'{result_dir}/{dataset}_{ea_ous}_seed{seed}.csv')
        df_epis_alea = pd.read_csv(f'{result_dir}/{dataset}_{ea}_seed{seed}.csv')
        df_epis = pd.read_csv(f'{result_dir}/{dataset}_{e}_seed{seed}.csv')

        calibration_data_epis_alea_ours_pred = df_epis_alea_ours['Predicted Probability'].values
        calibration_data_epis_alea_pred = df_epis_alea['Predicted Probability'].values
        calibration_data_epis_pred = df_epis['Predicted Probability'].values


        calibration_data_epis_alea_ours_obser = df_epis_alea_ours['Observed Probability'].values
        ea_ous_obsers[seed-1].extend(calibration_data_epis_alea_ours_obser.flatten())
        
        calibration_data_epis_alea_obser = df_epis_alea['Observed Probability'].values
        ea_obsers[seed-1].extend(calibration_data_epis_alea_obser.flatten())

        calibration_data_epis_obser = df_epis['Observed Probability'].values
        e_obsers[seed-1].extend(calibration_data_epis_obser.flatten())


    ea_ous_obsers = np.array(ea_ous_obsers)
    ea_obsers = np.array(ea_obsers)
    e_obsers = np.array(e_obsers)


    ea_ours_obsers_upper = np.mean(ea_ous_obsers, axis=0) + 1 * np.std(ea_ous_obsers, axis=0)
    ea_ours_obsers_lower = np.mean(ea_ous_obsers, axis=0) - 1 * np.std(ea_ous_obsers, axis=0)
    ea_obsers_upper = np.mean(ea_obsers, axis=0) + 1 * np.std(ea_obsers, axis=0)
    ea_obsers_lower = np.mean(ea_obsers, axis=0) - 1 * np.std(ea_obsers, axis=0)
    e_obsers_upper = np.mean(e_obsers, axis=0) + 1 * np.std(e_obsers, axis=0)
    e_obsers_lower = np.mean(e_obsers, axis=0) - 1 * np.std(e_obsers, axis=0)

    axes[row, col].fill_between(calibration_data_epis_alea_ours_pred, ea_ours_obsers_upper, ea_ours_obsers_lower, color='red', alpha=0.2)
    axes[row, col].fill_between(calibration_data_epis_alea_pred, ea_obsers_upper, ea_obsers_lower, color='blue', alpha=0.2)
    axes[row, col].fill_between(calibration_data_epis_pred, e_obsers_upper, e_obsers_lower, color='green', alpha=0.2)

    axes[row, col].plot([0, 1], [0, 1], linestyle='--', color='gray', alpha=1, label='perfectly calibrated')
    
    axes[row, col].plot(calibration_data_epis_alea_ours_pred, np.mean(ea_ous_obsers, axis=0), color='red', alpha=0.8, label='epis + alea (ous)', marker='o', linestyle='-')
    axes[row, col].plot(calibration_data_epis_alea_pred, np.mean(ea_obsers, axis=0), color='blue', alpha=0.8, label='epis + alea', marker='o', linestyle='-')
    axes[row, col].plot(calibration_data_epis_pred, np.mean(e_obsers, axis=0), color='green', alpha=0.8, label='epis', marker='o', linestyle='-')

   
    axes[row, col].set_xlabel('Predicted Probability')
    axes[row, col].set_ylabel('Observed Probability')
    axes[row, col].set_title(dataset)

plt.tight_layout()
plt.xticks([0.0, 0.2, 0.4, 0.6, 0.8, 1.0])
plt.yticks([0.00, 0.25, 0.50, 0.75, 1.00])
axes[0, 2].legend(loc='upper left', bbox_to_anchor=(1, 1))
plt.savefig('calibration.png', dpi=300, bbox_inches='tight')


In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

result_dir = '../result/calibration' # 'result_5_2'
ea_ous = 'epis_alea_ours'
ea = 'epis_alea'
e = 'epis'
dataset_list =  [ "SMAP_P1",  "NAB_Ambient", "NAB_Texi", "NAB_Machine",  "SMD_machine3-4", "MSL_F7", "MSL_P11", "SMAP_E13", "SMAP_E1", "UCR_InternalBleeding16", "UCR_InternalBleeding17", "SMD_machine2-1"]
 
for index, dataset in enumerate(dataset_list):
    ea_ous_obsers = [[] for i in range(5)]
    ea_obsers = [[] for i in range(5)]
    e_obsers = [[] for i in range(5)]

    exists = any(glob.glob(f'{result_dir}/{dataset}*'))
    if exists == False:
        print(f'{dataset} does not exist, skipping...')
        continue

    for seed in [1,2,3,4,5]:
        df_epis_alea_ours = pd.read_csv(f'./{result_dir}/{dataset}_{ea_ous}_seed{seed}.csv')
        df_epis_alea = pd.read_csv(f'./{result_dir}/{dataset}_{ea}_seed{seed}.csv')
        df_epis = pd.read_csv(f'./{result_dir}/{dataset}_{e}_seed{seed}.csv')

        calibration_data_pred = df_epis['Predicted Probability'].values


        calibration_data_epis_alea_ours_obser = df_epis_alea_ours['Observed Probability'].values
        ea_ous_obsers[seed-1].extend(calibration_data_epis_alea_ours_obser.flatten())
        
        calibration_data_epis_alea_obser = df_epis_alea['Observed Probability'].values
        ea_obsers[seed-1].extend(calibration_data_epis_alea_obser.flatten())

        calibration_data_epis_obser = df_epis['Observed Probability'].values
        e_obsers[seed-1].extend(calibration_data_epis_obser.flatten())



    ea_ous_obsers = np.array(ea_ous_obsers)
    ea_obsers = np.array(ea_obsers)
    e_obsers = np.array(e_obsers)
    mse_epis_mean = np.mean((np.mean(e_obsers, axis=0) - calibration_data_pred)**2)
    mse_epis_alea_mean = np.mean((np.mean(ea_obsers, axis=0) - calibration_data_pred)**2)
    mse_ours_mean = np.mean((np.mean(ea_ous_obsers, axis=0) - calibration_data_pred)**2)

    print(f"{dataset}:  mse_epis: {mse_epis_mean:.4f}, mse_epis_alea: {mse_epis_alea_mean:.4f}, mse_ours: {mse_ours_mean:.4f}")


SMAP_P1:  mse_epis: 0.1223, mse_epis_alea: 0.1000, mse_ours: 0.0956
NAB_Ambient does not exist, skipping...
NAB_Texi does not exist, skipping...
NAB_Machine does not exist, skipping...
SMD_machine3-4 does not exist, skipping...
MSL_F7 does not exist, skipping...
MSL_P11 does not exist, skipping...
SMAP_E13 does not exist, skipping...
SMAP_E1 does not exist, skipping...
UCR_InternalBleeding16 does not exist, skipping...
UCR_InternalBleeding17 does not exist, skipping...
SMD_machine2-1 does not exist, skipping...
